<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries" data-toc-modified-id="Libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Data-scraper-from-ESMI" data-toc-modified-id="Data-scraper-from-ESMI-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data scraper from ESMI</a></span></li><li><span><a href="#Process-time-series" data-toc-modified-id="Process-time-series-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Process time series</a></span><ul class="toc-item"><li><span><a href="#Voltage-time-series-for-the-south" data-toc-modified-id="Voltage-time-series-for-the-south-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Voltage time series for the south</a></span></li><li><span><a href="#Locations-for-the-data-points" data-toc-modified-id="Locations-for-the-data-points-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Locations for the data points</a></span></li></ul></li><li><span><a href="#Scratch-book" data-toc-modified-id="Scratch-book-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Scratch book</a></span></li></ul></div>

# Libraries

In [63]:
import csv
import numpy as np
import sys
from bs4 import BeautifulSoup
import os
from datetime import datetime
from datetime import time, tzinfo, timedelta

import requests
import pandas as pd
from tqdm import tqdm
import re
import pickle

# Data scraper from ESMI


In [ ]:
url_lists = []
# "Andhra Pradesh","Assam","Jharkhand","Karnataka","Maharashtra","Odisha" ,"Punjab","Tamil Nadu","Telangana" ,"Uttar Pradesh"

# 'http://www.watchyourpower.org/reports.php?category_id=3&state=Andhra Pradesh&district=Jorhat'
headers={
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
    'Content-Type': 'application/json'
    }
#payload= {"postType":"Both","day":"Day1095","sort":"InsOrRegDate","order":"Descending","size":20,"offset":"20","pageSource":"search"}

for c in [2,3,4,5]:
    api_url= f'http://www.watchyourpower.org/reports.php?category_id={c}'
    print(api_url)
    res=requests.post(api_url,headers=headers)
    if res. ok:
      text = res.text
      text_list = text.split('<option value="">-- Select  --</option>')[1].split('</select>')[0]
      text_list = re.split('<option class="" | >', text_list)
      text_list2 = [elem[6:].replace('\"', '').replace(' ', '+')  for elem in text_list if elem.startswith("value=")]
    print(c,text_list2)

    for id in text_list2:
        result = []
        for year in range(2004,2024):
            for month in range(1,13):

                next_month =  1 if month==12 else month+1
                next_year = year + 1 if month==12 else year
                url= f'http://www.watchyourpower.org/reports.php?category_id={c}&location_id={id}&from_date=13%2F{month_2dight[month]}%2F{year}&to_date=13%2F{month_2dight[next_month]}%2F{next_year}'
                print(url)

                # res=requests.post(url,headers=headers)
                try:
                    res=requests.post(url,headers=headers, timeout=5)
                except requests.Timeout:
                    # back off and retry
                    pass
                except requests.ConnectionError:
                    pass
                if res.ok:
                  print('ok', year, month)
                  text = res.text
                  title= text.split('<title>')[1].split('for')[0]
                  print(title)
                  text_list = text.split('\n')
                  for elem in text_list:
                      # print(type(elem),elem)
                      if elem.startswith("\t\tvar linechartData"):
                          # print(elem[24:-3])
                          voltage_list = elem[24:-3].split('},{')
                          # print(voltage_list)
                          for volt in tqdm(voltage_list):
                              # print(volt)
                              result.append(eval("{"+volt+"}"))
        print(len(result))
        save = {'name': title, 'timeseris':result}
        with open(f'{title}.pkl', 'wb') as f:
            pickle.dump(save, f)

# Process time series

## Voltage time series for the south

In [121]:
pkl_folder_dir = '../../save/'

# Check for visited files
save_folder_dir = '../../ESMI_csv/'
file_id_visited = []
for item in os.listdir(save_folder_dir):
    file_id_visited.append(int(item.split('_')[0]))

for item in tqdm(os.listdir(pkl_folder_dir)):
    if int(item.split('.')[0]) in file_id_visited:
        continue
    else:
        try:
            # Load file
            file = pickle.load(open(pkl_folder_dir + item, 'rb'))
            raw_df = pd.DataFrame(file)

            # Location name
            location_name = pd.DataFrame(file)['name'].unique()[0]
            state_name = location_name.split(' - ')[1].split(', ')[0]
            district_name = location_name.split(' - ')[1].split(', ')[1]
            location_name = location_name.split(' - ')[1].split(', ')[-1][:-1]
            # check '/' in names
            if '/' in state_name:
                state_name = state_name.replace("/", "_")
            if '/' in district_name:
                district_name = district_name.replace("/", "_")
            if '/' in location_name:
                location_name = location_name.replace("/", "_")

            # Extract datetime and voltage data
            df = raw_df[['timeseris']]
            df['datetime'] = df.apply(lambda row: datetime.strptime(row['timeseris']['date'], '%a %b %d %Y %H:%M:%S'), axis=1)
            df['voltage'] = df.apply(lambda row: row['timeseris']['voltage'], axis=1)

            # Save dataframe to csv file
            df.to_csv(f'../../ESMI_csv/{int(item[:-4])}_{state_name}_{district_name}_{location_name}.csv')
        except FileNotFoundError:
            print(item)


100%|██████████| 43/43 [33:22<00:00, 46.57s/it]


## Locations for the data points

# Scratch book